# Bibliotecas utilizadas



In [1]:
import cv2
import numpy as np
import math
from matplotlib import pyplot as plt
#vgg3.py
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt; import numpy as np

# Horizontalização da imagem.

In [2]:
#alinha
def calc_angs(img_gsc):
    m = cv2.moments(img_gsc)
    cen_x = m['m10'] / m['m00']
    cen_y = m['m01'] / m['m00']
    theta = 0.5 * math.atan2(2 * m['mu11'], m['mu20'] - m['mu02'])
    rotacao = cv2.getRotationMatrix2D((cen_x, cen_y), theta*360/(2*3.14159), 1.0)
    rotacionado = cv2.warpAffine(img_gsc, rotacao, (largura, altura))
    return rotacionado


direct_root = '/content/drive/MyDrive/Mestrado/Colab Codes/Deep Learning/rice_1to1000/'
direct_arborio = 'Arborio/Arborio ('
direct_basmati = 'Basmati/Basmati ('
direct_ipsala = 'Ipsala/Ipsala ('
direct_jasmine = 'Jasmine/Jasmine ('
direct_karacadag='Karacadag/Karacadag ('

directs_data = [direct_arborio,direct_basmati,direct_ipsala,direct_jasmine,direct_karacadag]
escolha=3
for i in range(1,1001):
  #print(direct_root + directs_data[escolha]+str(i)+').jpg')
  img_orig = cv2.imread(direct_root + directs_data[escolha]+str(i)+').jpg')
  img_gsc = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
  ret, img_gsc = cv2.threshold(img_gsc, 127, 255, cv2.THRESH_BINARY)
  if (img_orig is None):
    print("img inexistente para i="+str(i))
    break
  else:
    largura, altura = np.shape(img_gsc)
   # img_saida = calc_angs(img_gsc)
    cv2.imwrite(direct_root+directs_data[escolha]+str(i)+')pb.png',img_gsc)



plt.imshow( img_gsc,cmap="gray",interpolation="nearest")
plt.show()
plt.imshow( img_gsc,cmap="gray",interpolation="nearest")
plt.show()

KeyboardInterrupt: ignored

# Treinamento da rede neural, considerando ax como imagnes de arroz e ay como vetor de 5 posicoes [1,1,1,1,1].

In [ ]:
print("Iniciar movimentacao ax ay ##############")

direct_root = '/content/drive/MyDrive/Mestrado/Colab Codes/Deep Learning/rice_1to1000/'
direct_arborio = "Arborio/Arborio ("
direct_basmati = "Basmati/Basmati ("
direct_ipsala = "Ipsala/Ipsala ("
direct_jasmine = "Jasmine/Jasmine ("
direct_karacadag="Karacadag/Karacadag ("

directs_data = [direct_arborio,direct_basmati,direct_ipsala,direct_jasmine,direct_karacadag]
escolha=4
particao=599
counterq=-1
countera=-1
size=64
ax = np.zeros([5*particao,size,size],dtype=np.uint32)
ay=np.zeros([5*particao,1],dtype=np.uint32)
qx=np.zeros([5000-5*particao,size,size],dtype=np.uint32)
qy=np.zeros([5000-5*particao,1],dtype=np.uint32)
for  tipo  in directs_data:
    for i in range(1,1000):
    # regra de particao de 100 pra treino ax ay
      if (i <particao):
        countera+=1
      #  print("  ---tipo: "+ str(tipo) + "Contador a: "+str(countera))
        ax[countera]=cv2.resize(cv2.imread(str(direct_root) + str(tipo)+str(i)+').png',cv2.IMREAD_GRAYSCALE),(size,size))
        if (tipo==direct_arborio):
          ay[countera]=[0]
        else:
          if(tipo==direct_basmati):
           ay[countera]=[1]
          else:
            if(tipo==direct_ipsala):
             ay[countera]=[2]
            else:
             if(tipo==direct_jasmine):
                ay[countera]=[3]
             else:
                if(tipo==direct_karacadag):
                  ay[countera]=[4]
       # print("---- amostra treino- tipo: "+ str(tipo) +"  Valor tipo"+ str(ay[countera]) +" Contador a: "+str(countera))
      else:
       # print("tipo: "+ str(tipo) + "Contador a: "+str(counterq))
        counterq+=1
        qx[counterq]=cv2.resize(cv2.imread(str(direct_root) + str(tipo)+str(i)+').png',cv2.IMREAD_GRAYSCALE),(size,size))
        if (tipo==direct_arborio):
          qy[counterq]=[0]
        else:
         if(tipo==direct_basmati):
           qy[counterq]=[1]
         else:
            if(tipo==direct_ipsala):
              qy[counterq]=[2]
            else:
              if(tipo==direct_jasmine):
                qy[counterq]=[3]
              else:
                if(tipo==direct_karacadag):
                  qy[counterq]=[4]
       # print("xxxxxxxx amostra qx tipo: "+ str(tipo) +" Valor tipo:"+ str(qy[counterq]) +" Contador a: "+str(counterq))
    #print(direct_root + directs_data[escolha]+str(i)+').jpg')
print(np.shape(ax))
print(np.shape(ay))
print(np.shape(qx))
print(np.shape(qy))
print("Fim "+ str(np.shape(ax)))
#ay.transpose()
print(np.shape(ay))


ax=np.expand_dims(ax,axis=-1)
qx=np.expand_dims(qx,axis=-1)
print("Depois da insercao ax: " + str(np.shape(ax)))
print("Depois da insercao qx: " + str(np.shape(qx)))
print("Amostra ay" + str(ay[1]))

def impHistoria(history):
  print(history.history.keys())
  plt.plot(history.history['accuracy']); plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy'); plt.ylabel('accuracy'); plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left'); plt.show()
  plt.plot(history.history['loss']); plt.plot(history.history['val_loss'])
  plt.title('model loss'); plt.ylabel('loss'); plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left'); plt.show()

batch_size = 50; num_classes = 5; epochs = 200
nl, nc = size,size; input_shape = (nl, nc,1)
#(ax, ay), (qx, qy) = cifar10.load_data()

ax = ax.astype('float32'); ax /= 255 #0 a 1
qx = qx.astype('float32'); qx /= 255 #0 a 1

#ay=np.array(np.array_split(ay,1))

ay = keras.utils.to_categorical(ay, num_classes)
print(np.shape(ay))
qy = keras.utils.to_categorical(qy, num_classes)
print(np.shape(qy))
print("depois do keras")
print(np.shape(ax))
print(np.shape(ay))
print(np.shape(qx))
print(np.shape(qy))

kweight=5e-4; bweight=5e-4;

model = Sequential()
model.add(Conv2D(20, kernel_size=(5,5), activation='relu', input_shape=input_shape,kernel_regularizer=l2(kweight),bias_regularizer=l2(bweight))) #nx28x28x20
model.add(MaxPooling2D(pool_size=(2,2))) #nx14x14x20
model.add(Conv2D(40, kernel_size=(5,5), activation='relu',kernel_regularizer=l2(kweight),bias_regularizer=l2(bweight))) #nx10x10x40
model.add(MaxPooling2D(pool_size=(2,2))) #nx5x5x40
model.add(Flatten()) #nx1000
model.add(Dropout(0.25))
model.add(Dense(1000, activation='relu')) #nx1000
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax')) #nx10

from tensorflow.keras.utils import plot_model
plot_model(model, to_file='vgg4.png', show_shapes=True); model.summary()

datagen = ImageDataGenerator(
  featurewise_center=False,  # set input mean to 0 over the dataset
  samplewise_center=False,  # set each sample mean to 0
  featurewise_std_normalization=False,  # divide inputs by std of the dataset
  samplewise_std_normalization=False,  # divide each input by its std
  zca_whitening=False,  # apply ZCA whitening
  rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
  width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
  height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
  horizontal_flip=True,  # randomly flip images
  vertical_flip=False)  # randomly flip images
datagen.fit(ax)

reduce_lr = ReduceLROnPlateau(monitor='accuracy',
factor=0.9, patience=2, min_lr=0.0000000001, verbose=True)

opt=optimizers.Adam()

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit(datagen.flow(ax,ay,batch_size=batch_size),steps_per_epoch=ax.shape[0]//batch_size,epochs=epochs, verbose=2, validation_data=(qx, qy),callbacks=[reduce_lr])

impHistoria(history)

score = model.evaluate(qx, qy, verbose=0)
saida=model.pre
print('Test loss:', score[0]); print('Test accuracy:', score[1])
model.save('vgg4.h5')


In [ ]:
from matplotlib import pyplot as plt
(ax, ay), (qx, qy) = cifar10.load_data()
ax = ax.astype('float32'); ax /= 255 #0 a 1
qx = qx.astype('float32'); qx /= 255 #0 a 1
print("antes")
print(np.shape(ax))
print(np.shape(ay))
print(ay[3])
print(np.shape(qx))
print(np.shape(qy))
ay = keras.utils.to_categorical(ay, 10)
qy = keras.utils.to_categorical(qy, 10)
print("depois")
print(np.shape(ax))
print(np.shape(ay))
print(np.shape(qx))
print(np.shape(qy))

print(qy[1])
a=np.array([1, 1,1, 1, 1])

print("antes: " + str(np.shape(a)))

print(ax[1])
a1=np.array_split(a,1)
print(np.shape(a1))
plt.imshow(ax[1][2])
plt.show()

In [ ]:
direct_root = '/content/drive/MyDrive/Mestrado/Colab Codes/Deep Learning/rice_1to1000/'
direct_arborio = "Arborio/Arborio ("
direct_basmati = "Basmati/Basmati ("
direct_ipsala = "Ipsala/Ipsala ("
direct_jasmine = "Jasmine/Jasmine ("
direct_karacadag="Karacadag/Karacadag ("

directs_data = [direct_arborio,direct_basmati,direct_ipsala,direct_jasmine,direct_karacadag]#,direct_basmati,direct_ipsala,direct_jasmine,direct_karacadag]
escolha=4
ax = np.zeros([100,128,128],dtype=np.float32)
ay=np.zeros([100,5],dtype=np.float32)
qx=np.zeros([900,32,32],dtype=np.float32)
qy=np.zeros([900,5],dtype=np.float32)
cot=0

for  tipo  in directs_data:
 # print(tipo)
  for i in range(1,1001):
    if(tipo==direct_arborio):
      a=1
    # regra de particao de 100 pra treino ax ay
    if (i <=2):
      ax[cot]= cv2.resize(cv2.imread(str(direct_root) + str(tipo)+str(i)+').png',cv2.IMREAD_UNCHANGED),(128,128))
      a1=cv2.resize(cv2.imread(str(direct_root) + str(tipo)+str(i)+').png',cv2.IMREAD_UNCHANGED),(128,128))
      cot+=1
      print(str(direct_root) + str(tipo)+str(i)+').png')
     # plt.imshow(a1)
     # plt.show()

a1=cv2.imread('/content/drive/MyDrive/Mestrado/Colab Codes/Deep Learning/rice_1to1000/Arborio/Arborio (1).png',cv2.IMREAD_GRAYSCALE)
print(np.amax(ax[1]))
plt.imshow(ax[1], cmap='gray')
plt.show()